In [ ]:
!pip install giotto-tda
!pip install ripser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18101 sha256=8962ab6930b6bae572825ffc7d8a328bf32749a679a157240abf0eb81d8a25b7
  Stored in directory: /root/.cache/pip/wheels/ef/0f/3b/0f931844eecc34addd90e72d54cd39c08b7066c5f25c00b9a4
Successfully built hopcroftkarp


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# from gene_load_data0 import * #14classes


#from gene_load_data2 import * #7classes
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

#from torch.autograd import Variable as V


from sklearn.preprocessing import StandardScaler, MinMaxScaler

from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram, plot_heatmap, plot_point_cloud
from gtda.diagrams import PersistenceEntropy, PersistenceImage, PersistenceLandscape, Scaler, Filtering, PairwiseDistance, Amplitude
import plotly.express as px

from gtda.time_series import PearsonDissimilarity
from PIL import Image

from gtda.time_series import SingleTakensEmbedding, TakensEmbedding

import matplotlib
import matplotlib.pyplot as plt


import numpy as np
from ripser import Rips
from ripser import ripser
from persim import plot_diagrams, PersistenceImager
import persim

from persim.landscapes import (
    PersistenceLandscaper,
    PersLandscapeExact,
    PersLandscapeApprox,
    average_approx,
    snap_pl,
    plot_landscape,
    plot_landscape_simple
)

from persim.persistent_entropy import *
import ripser

# Import Scikit-Learn tools
from sklearn.decomposition import PCA
from numpy import inf

import time
import datetime
from google.colab.patches import cv2_imshow
import scipy.io as sio
import os
from scipy import sparse
from persim import PersImage
import sys
from ripser import ripser
import cv2 as cv2
from skimage.transform import resize
from sklearn.decomposition import PCA

In [ ]:
def gen_PI_image(img, threshold = 0.02, pixel = 50, spreadval = 0.01, rangemin = 0.0, rangemax = 0.3, out_path = ''):
  pim = PersImage(pixels=[pixel,pixel], spread=spreadval, specs={"minBD": rangemin, "maxBD": rangemax})
  out_img = np.zeros((pixel,pixel,3))
  for i in range(3):
    x_data = img[:,:,i]
    # x_data.T # TO GENERATE COL WISE PI Images
    x = x_data.reshape((-1))
    x = x/255.0

    # Extracting Needed Info
    N = len(x)
    t = np.arange(N)

    #Sublevelset Filtration
    # Add edges between adjacent points in the time series, with the "distance"
    # along the edge equal to the max value of the points it connects
    I = np.arange(N-1)
    J = np.arange(1, N)
    V = np.maximum(x[0:-1], x[1::])

    # Add vertex birth times along the diagonal of the distance matrix
    I = np.concatenate((I, np.arange(N)))
    J = np.concatenate((J, np.arange(N)))
    V = np.concatenate((V, x))

    #Create the sparse distance matrix
    D = sparse.coo_matrix((V, (I, J)), shape=(N, N)).tocsr()
    dgm0 = ripser(D, maxdim=0, distance_matrix=True)['dgms'][0]
    dgm0 = dgm0[dgm0[:, 1]-dgm0[:, 0] > 1e-3, :]
    allgrid = np.unique(dgm0.flatten())
    allgrid = allgrid[allgrid < np.inf]
    xs = np.unique(dgm0[:, 0])
    ys = np.unique(dgm0[:, 1])
    ys = ys[ys < np.inf]

    # Removing Infinity Points
    where_are_inf = np.isinf(dgm0)
    dgm0 = dgm0[~where_are_inf[:,1]]

    # Computing Lifetimes for Thresholding of Persistence Diagrams
    lifetime = np.ones(len(dgm0))

    for h in range(len(dgm0)):
      lifetime[h] = dgm0[h][1] - dgm0[h][0]

    #Keeps only points greater than reduction value
    isvalid = np.greater(lifetime, threshold)
    reduced_pd = dgm0[isvalid]

    imgs = pim.transform(reduced_pd)
    # print(np.max(imgs))
    if np.max(imgs) > 0:
      imgs = imgs/np.max(imgs)
    imgs = (imgs* 255).astype('uint8')
    
    out_img[:,:,i] = imgs

  cv2.imwrite(out_path, out_img)


In [ ]:
data_dir = 'Data/cifar10'

In [ ]:
for f in os.listdir(data_dir):

  if os.path.isdir(os.path.join(data_dir, f)):
    sub_data_dir = os.path.join(data_dir, f)
    out_dir = os.path.join(data_dir, f+'_PI')

    if not os.path.exists(out_dir):
      os.makedirs(out_dir)
    print(f"------Working on {f}ing data-----------")
    for sf in os.listdir(sub_data_dir):
      input_file_path = os.path.join(sub_data_dir, sf)
      out_file_path = os.path.join(out_dir, sf)
      if not os.path.exists(out_file_path):
        os.makedirs(out_file_path)
      print(f"Data: {sf}")
      file_name = os.listdir(input_file_path)
      for fn in tqdm(range(len(file_name))):
        img = cv2.imread(os.path.join(input_file_path, file_name[fn]))
        gen_PI_image(img, out_path=os.path.join(out_file_path, file_name[fn]))